# Import Libraries

In [1]:
!pip install -q --upgrade "tensorflow<2.16" "tf-keras<2.16"

In [2]:
!pip install langdetect

In [3]:
!pip install googletrans==4.0.0-rc1

In [4]:
!pip install pysastrawi

In [3]:
import re
import numpy as np
import pandas as pd
import nltk
import time
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from googletrans import Translator
from langdetect import detect
from sentence_transformers import SentenceTransformer
nltk.download('stopwords')
nltk.download('punkt_tab')
from sklearn.metrics.pairwise import cosine_similarity
import shutil

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Data Loading (Cleaned Data)


In [68]:
#load data from CSV file
df = pd.read_csv('file_bersih_final_project.csv')
df

,Unnamed: 0,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10
0,0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,NaN,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,"""Fatale"" adalah aroma sensual dan intens yang ...","Wanginya enak, laki juga masuk rekomendasii ju...","Wanginya enak, laki juga masuk rekomendasii ju...",HINT Fatale Extrait de Parfum benar-benar biki...,"Wanginya enak tahan lama, gak bikin bau apek, ...",Wanginya sangat seductivee dan kenceng banvwt....,"wanginya enak bgt tahan lama, very recommended...","wanginya super wangi poll, gabikin pusing di k...",wanginya enak banget bikin seger dan ga bikin ...
1,1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,NaN,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,Udah gatau botol keberapa karena selalu repurc...,Produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...,parfum kesukaan ibu akuu. wanginya enak trus l...,"Suka sama wanginya! Musky, powder, floral lemb...","Wanginya enak bgt, soft gitu, ga bikin pusing....",Sukaa si sama aroma nya white musk kalo kelama...,"aku punya yg versi roll on, jadi kayak oil yg ...",Sangaaaatttt suka!!! Varian yang vegan ini aku...
2,2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,NaN,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...","It’s just tooo much for me, buat yang suka par...",Parfum yang sangat amat hitz di semua kalangan...,Parfume yang wangi nya dar der dor & parfume k...,Cobain parfume dari saff & co varian SOTB ini ...,"Baru coba saff & co edp sotb nya yg viral, aku...","Top Notes : Mandarin, Galbanum, Ylang",Best parfum kala kalian suka dengan tipe wangi...,"saff n co ini SPL nya oke juga, juice parfumny...","Wangi yang bikin orang-orang auto pada nengok,..."
3,3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,NaN,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,Ini koleksi parfume HMNS ke-2 ku. Pertama kali...,Walopun aku cuma coba testernya tapi parfum ya...,Pertama kali tau parfum ini dari kakak aku. Ak...,"Pertama kali coba HMNS ya si Orgasm ini, awal ...","Pertama kali cobain parfum nya hmns, dan banya...",baunya skrg lebih enak drpd dulu waktu awal la...,Parfume favorite aku 😩 wanginya tu bunga gitu ...,"I got this as a gift from my friend, pernah co...","best varian ever, suka banhet sama wangi nya s..."
4,4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,"wanginya enak banget parfum ini,menurut gua ke...",pertama kali cium langsung suka sama wanginya😍...,"Ini parfum sejuta umat sih, memang wanginya en...","Dia tuh bau nya manis seger tapi strong gitu, ...","wanginya enak banget menurut aku, wanginya seg...",Saya beli parfum ini dengan harga yang sangat ...,Varian parfum evangeline ini yang paling norma...,Black sakura wanginya unik banget soalnya dari...,"hai girls! beli ini karna racun banget, +harga..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,Crusita,Tea Collection Extrait De Parfum,Genmaicha,Tea Collection Extrait De Parfum Genmaicha,5.0,4,210000,Genmaicha ini menurut aku pertam

Dari data diatas terdapat kolom unnamed. Ini akan di remove karena duplikat

In [69]:
#remove column Unnamed: 0
df.drop(columns='Unnamed: 0',inplace=True)
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,NaN,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,"""Fatale"" adalah aroma sensual dan intens yang ...","Wanginya enak, laki juga masuk rekomendasii ju...","Wanginya enak, laki juga masuk rekomendasii ju...",HINT Fatale Extrait de Parfum benar-benar biki...,"Wanginya enak tahan lama, gak bikin bau apek, ...",Wanginya sangat seductivee dan kenceng banvwt....,"wanginya enak bgt tahan lama, very recommended...","wanginya super wangi poll, gabikin pusing di k...",wanginya enak banget bikin seger dan ga bikin ...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,NaN,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,Udah gatau botol keberapa karena selalu repurc...,Produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...,parfum kesukaan ibu akuu. wanginya enak trus l...,"Suka sama wanginya! Musky, powder, floral lemb...","Wanginya enak bgt, soft gitu, ga bikin pusing....",Sukaa si sama aroma nya white musk kalo kelama...,"aku punya yg versi roll on, jadi kayak oil yg ...",Sangaaaatttt suka!!! Varian yang vegan ini aku...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,NaN,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...","It’s just tooo much for me, buat yang suka par...",Parfum yang sangat amat hitz di semua kalangan...,Parfume yang wangi nya dar der dor & parfume k...,Cobain parfume dari saff & co varian SOTB ini ...,"Baru coba saff & co edp sotb nya yg viral, aku...","Top Notes : Mandarin, Galbanum, Ylang",Best parfum kala kalian suka dengan tipe wangi...,"saff n co ini SPL nya oke juga, juice parfumny...","Wangi yang bikin orang-orang auto pada nengok,..."
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,NaN,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,Ini koleksi parfume HMNS ke-2 ku. Pertama kali...,Walopun aku cuma coba testernya tapi parfum ya...,Pertama kali tau parfum ini dari kakak aku. Ak...,"Pertama kali coba HMNS ya si Orgasm ini, awal ...","Pertama kali cobain parfum nya hmns, dan banya...",baunya skrg lebih enak drpd dulu waktu awal la...,Parfume favorite aku 😩 wanginya tu bunga gitu ...,"I got this as a gift from my friend, pernah co...","best varian ever, suka banhet sama wangi nya s..."
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,"wanginya enak banget parfum ini,menurut gua ke...",pertama kali cium langsung suka sama wanginya😍...,"Ini parfum sejuta umat sih, memang wanginya en...","Dia tuh bau nya manis seger tapi strong gitu, ...","wanginya enak banget menurut aku, wanginya seg...",Saya beli parfum ini dengan harga yang sangat ...,Varian parfum evangeline ini yang paling norma...,Black sakura wanginya unik banget soalnya dari...,"hai girls! beli ini karna racun banget, +harga..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,Crusita,Tea Collection Extrait De Parfum,Genmaicha,Tea Collection Extrait De Parfum Genmaicha,5.0,4,210000,Genmaicha ini menurut aku pertama spray kerasa...,parfum genma

In [70]:
#check data type and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   link_product       1020 non-null   object 
 1   product_image      1020 non-null   object 
 2   brand_name         1020 non-null   object 
 3   product_name       1020 non-null   object 
 4   product_variant    379 non-null    object 
 5   full_name_product  1020 non-null   object 
 6   ratings            1020 non-null   float64
 7   total_review       1020 non-null   int64  
 8   price              1020 non-null   int64  
 9   review_1           1020 non-null   object 
 10  review_2           1020 non-null   object 
 11  review_3           1019 non-null   object 
 12  review_4           1019 non-null   object 
 13  review_5           922 non-null    object 
 14  review_6           807 non-null    object 
 15  review_7           733 non-null    object 
 16  review_8           662 n

Kolom product_variant, dan beberapa kolom review ada null value. Ini nanti akan di handle

Kolom product_variant adalah extensi dari kolom product_name. Ini sudah dijadikan 1 kesatuan pada kolom full_name_product. Akan kita lihat perbedaan dari ketiga kolom ini



In [71]:
df[['product_name','product_variant','full_name_product']]

,product_name,product_variant,full_name_product
0,Fatale Extrait de Parfum,NaN,Fatale Extrait de Parfum
1,White Musk Eau de Parfum,NaN,White Musk Eau de Parfum
2,S.O.T.B Extrait de Parfum,NaN,S.O.T.B Extrait de Parfum
3,Orgasm,NaN,Orgasm
4,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura
...,...,...,...
1015,Tea Collection Extrait De Parfum,Genmaicha,Tea Collection Extrait De Parfum Genmaicha
1016,Ayodhya EDP,NaN,Ayodhya EDP
1017,Eau Capitale,NaN,Eau Capitale
1018,Maidens Prayer Eau de Parfum,Summer Phoenix,Maidens Prayer Eau de Parfum Summer Phoenix


Dari data diatas, dapat kita lihat bahwa kolom full_name_product merupakan hasil concat dari product_name dan product_variant. Maka dari itu, kolom yang digunakan untuk modelling adalah kolom full_name_product

In [72]:
#fill missing values with -
df.fillna('-',inplace=True)

In [73]:
#check null value
df.isnull().sum()

,0
link_product,0
product_image,0
brand_name,0
product_name,0
product_variant,0
full_name_product,0
ratings,0
total_review,0
price,0
review_1,0


# Data Processing

Melakukan penghapusan stopwords, emoji, hashstag, url.

Translate full english sentences menjadi Indonesian

stemming+tokenization

In [7]:
#create stopwords
stopwords_en = stopwords.words("english")
stopwords_id = list(set(stopwords.words('indonesian')))
stopwords_id.append('oh')

In [8]:
#create stemmer
stemmer_id = StemmerFactory().create_stemmer()
stemmer_en = PorterStemmer()

In [76]:
def text_preprocessing_basic(text):
  # Case folding
  text = text.lower()

  # Mention removal
  text = re.sub("@[A-Za-z0-9_]+", " ", text)

  # Hashtags removal
  text = re.sub("#[A-Za-z0-9_]+", " ", text)

  # Newline removal (\n)
  text = re.sub(r"\\n", " ",text)

  # Whitespace removal
  text = text.strip()

  # URL removal
  text = re.sub(r"http\S+", " ", text)
  text = re.sub(r"www.\S+", " ", text)

  # Non-letter removal (such as emoticon, symbol (like μ, $, 兀), etc
  text = re.sub("[^A-Za-z\s']", " ", text)
  text = text.strip()
  return text

In [77]:
for i in range(1,11):
  df[f'review_{i}_processed'] = df[f'review_{i}'].apply(lambda x: text_preprocessing_basic(x))

Karena ada kemungkinan pada hasil basic text processing ada data yang tidak memiliki text, maka dari itu akan di replace menjadi missing value yang kemudian akan di handle

In [83]:
#replace row yang tidak memiliki text menjadi missing value
df.replace('', np.nan,inplace=True)

In [84]:
#cek apakah terdapat missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   link_product         1020 non-null   object 
 1   product_image        1020 non-null   object 
 2   brand_name           1020 non-null   object 
 3   product_name         1020 non-null   object 
 4   product_variant      1020 non-null   object 
 5   full_name_product    1020 non-null   object 
 6   ratings              1020 non-null   float64
 7   total_review         1020 non-null   int64  
 8   price                1020 non-null   int64  
 9   review_1             1020 non-null   object 
 10  review_2             1020 non-null   object 
 11  review_3             1020 non-null   object 
 12  review_4             1020 non-null   object 
 13  review_5             1020 non-null   object 
 14  review_6             1020 non-null   object 
 15  review_7             1020 non-null   o

Karena benar ada missing value, maka akan di handle

In [85]:
df.fillna("-",inplace = True)
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_1_processed,review_2_processed,review_3_processed,review_4_processed,review_5_processed,review_6_processed,review_7_processed,review_8_processed,review_9_processed,review_10_processed
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,dari awal semprot udah kerasa beda ada manisn...,fatale adalah aroma sensual dan intens yang c...,wanginya enak laki juga masuk rekomendasii ju...,wanginya enak laki juga masuk rekomendasii ju...,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,damn dulu aku cium parfum ini slalu kek gaksuk...,udah gatau botol keberapa karena selalu repurc...,produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,s o t b mengusung wangi citrus floral hangat ...,it s just tooo much for me buat yang suka par...,parfum yang sangat amat hitz di semua kalangan...,parfume yang wangi nya dar der dor parfume k...,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,top notes mandarin galbanum ylang,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,aku beli perfume ini karena penasaran kaya gim...,ini koleksi parfume hmns ke ku pertama kali...,walopun aku cuma coba testernya tapi parfum ya...,pertama kali tau parfum ini dari kakak aku ak...,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,varian parfum evangeline yang paling aku suka ...,wanginya enak banget parfum ini menurut gua ke...,pertama kali cium langsung suka sama wanginya ...,ini parfum sejuta umat sih memang wanginya en...,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [78]:
#check hasil handling
df.isnull().sum()

,0
link_product,0
product_image,0
brand_name,0
product_name,0
product_variant,0
full_name_product,0
ratings,0
total_review,0
price,0
review_1,0


In [86]:
#translate reviews to indonesian language
translator = Translator()
for n in range(1,5):
  ls = []
  for i in df[f'review_{n}_processed']:
      try:
        detect(i)
      except:
        ls.append(i)
      else:
        try:
          translator.translate(i,dest='id')
        except:
          ls.append(i)
        else:
          if(detect(i)!='id'):
            text = translator.translate(i,dest='id').text
            ls.append(text)
          else:
            ls.append(i)
          time.sleep(1)
  df[f'review_{n}_processed_tl'] = ls
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_5_processed,review_6_processed,review_7_processed,review_8_processed,review_9_processed,review_10_processed,review_1_processed_tl,review_2_processed_tl,review_3_processed_tl,review_4_processed_tl
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...,dari awal semprot udah kerasa beda ada manisn...,fatale adalah aroma sensual dan intens yang c...,wanginya enak laki juga masuk rekomendasii ju...,wanginya enak laki juga masuk rekomendasii ju...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...,damn dulu aku cium parfum ini slalu kek gaksuk...,udah gatau botol keberapa karena selalu repurc...,produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,top notes mandarin galbanum ylang,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...,s o t b mengusung wangi citrus floral hangat ...,it s just tooo much for me buat yang suka par...,parfum yang sangat amat hitz di semua kalangan...,parfume yang wangi nya dar der dor parfume k...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...,aku beli perfume ini karena penasaran kaya gim...,ini koleksi parfume hmns ke ku pertama kali...,walopun aku cuma coba testernya tapi parfum ya...,pertama kali tau parfum ini dari kakak aku ak...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...,varian parfum evangeline yang paling aku suka ...,wanginya enak banget parfum ini menurut gua ke...,pertama kali cium langsung suka sama wanginya ...,ini parfum sejuta umat sih memang wanginya en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [99]:
#Share Workload to other member(s)
df1 = df[['review_5_processed','review_6_processed','review_7_processed']]
df2 = df[['review_8_processed','review_9_processed','review_10_processed']]

df1.to_csv('translate_me.csv',index=False)
df2.to_csv('translate_me1.csv',index=False)

In [87]:
#save progress
df.to_csv('current.csv',index=False)

In [9]:
df = pd.read_csv('current.csv')
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_5_processed,review_6_processed,review_7_processed,review_8_processed,review_9_processed,review_10_processed,review_1_processed_tl,review_2_processed_tl,review_3_processed_tl,review_4_processed_tl
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...,dari awal semprot udah kerasa beda ada manisn...,fatale adalah aroma sensual dan intens yang c...,wanginya enak laki juga masuk rekomendasii ju...,wanginya enak laki juga masuk rekomendasii ju...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...,damn dulu aku cium parfum ini slalu kek gaksuk...,udah gatau botol keberapa karena selalu repurc...,produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,top notes mandarin galbanum ylang,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...,s o t b mengusung wangi citrus floral hangat ...,it s just tooo much for me buat yang suka par...,parfum yang sangat amat hitz di semua kalangan...,parfume yang wangi nya dar der dor parfume k...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...,aku beli perfume ini karena penasaran kaya gim...,ini koleksi parfume hmns ke ku pertama kali...,walopun aku cuma coba testernya tapi parfum ya...,pertama kali tau parfum ini dari kakak aku ak...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...,varian parfum evangeline yang paling aku suka ...,wanginya enak banget parfum ini menurut gua ke...,pertama kali cium langsung suka sama wanginya ...,ini parfum sejuta umat sih memang wanginya en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [18]:
#laod data from other member
df1 = pd.read_csv('translate_me_clean.csv')
df1

,Unnamed: 0,review_5_processed,review_6_processed,review_7_processed,review5_processed_tl,review6_processed_tl,review7_processed_tl
0,0,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...
1,1,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...
2,2,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,top notes mandarin galbanum ylang,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,Notet teratas juga minum
3,3,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...
4,4,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...
...,...,...,...,...,...,...,...
1015,1015,-,-,-,-,-,-
1016,1016,-,-,-,-,-,-
1017,1017,-,-,-,-,-,-
1018,1018,-,-,-,-,-,-


yang diambil hanya kolom yang sudah ter-translate. maka dari itu perlu di-rename agar sama dengan kolom df utama

In [20]:
#ambil kolom yang digunakan
df1 = df1[['review5_processed_tl','review6_processed_tl','review7_processed_tl']]
#rename column agar sama dengan df utama
df1.rename(columns={'review5_processed_tl':'review_5_processed_tl','review6_processed_tl':'review_6_processed_tl','review7_processed_tl':'review_7_processed_tl'},inplace=True)
df1

/tmp/ipython-input-20-2203125340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'review5_processed_tl':'review_5_processed_tl','review6_processed_tl':'review_6_processed_tl','review7_processed_tl':'review_7_processed_tl'},inplace=True)


,review_5_processed_tl,review_6_processed_tl,review_7_processed_tl
0,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...
1,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...
2,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,Notet teratas juga minum
3,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...
4,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...
...,...,...,...
1015,-,-,-
1016,-,-,-
1017,-,-,-
1018,-,-,-


In [17]:
#load data from other member
df2 = pd.read_csv('translate_me1_done.csv')
df2

,review_8_processed,review_9_processed,review_10_processed,review8_processed_tl,review9_processed_tl,review10_processed_tl
0,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...
1,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...
2,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...
3,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...
4,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...
...,...,...,...,...,...,...
1015,-,-,-,-,-,-
1016,-,-,-,-,-,-
1017,-,-,-,-,-,-
1018,-,-,-,-,-,-


kolom yang digunakan perlu direname agar sama dengan kolom pada dataframe utama

In [19]:
#ambil kolom yang digunakan
df2 = df2[['review8_processed_tl','review9_processed_tl','review10_processed_tl']]
df2.rename(columns={'review8_processed_tl':'review_8_processed_tl','review9_processed_tl':'review_9_processed_tl','review10_processed_tl':'review_10_processed_tl'},inplace=True)
df2

/tmp/ipython-input-19-14918965.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'review8_processed_tl':'review_8_processed_tl','review9_processed_tl':'review_9_processed_tl','review10_processed_tl':'review_10_processed_tl'},inplace=True)


,review_8_processed_tl,review_9_processed_tl,review_10_processed_tl
0,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...
1,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...
2,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...
3,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...
4,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...
...,...,...,...
1015,-,-,-
1016,-,-,-
1017,-,-,-
1018,-,-,-


In [12]:
#concate to current df
df = df.join(df1,lsuffix='_caller',how='left')
df = df.join(df2,lsuffix='_caller',how='left')
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_1_processed_tl,review_2_processed_tl,review_3_processed_tl,review_4_processed_tl,review_5_processed_tl,review_6_processed_tl,review_7_processed_tl,review_8_processed_tl,review_9_processed_tl,review_10_processed_tl
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,dari awal semprot udah kerasa beda ada manisn...,fatale adalah aroma sensual dan intens yang c...,wanginya enak laki juga masuk rekomendasii ju...,wanginya enak laki juga masuk rekomendasii ju...,hint fatale extrait de parfum benar benar biki...,wanginya enak tahan lama gak bikin bau apek ...,wanginya sangat seductivee dan kenceng banvwt ...,wanginya enak bgt tahan lama very recommended...,wanginya super wangi poll gabikin pusing di k...,wanginya enak banget bikin seger dan ga bikin ...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,damn dulu aku cium parfum ini slalu kek gaksuk...,udah gatau botol keberapa karena selalu repurc...,produk yang vegan dan alkohol free itu yang pa...,awalnya iseng mampir tokonya kecium aroma powd...,parfum kesukaan ibu akuu wanginya enak trus l...,suka sama wanginya musky powder floral lemb...,wanginya enak bgt soft gitu ga bikin pusing ...,sukaa si sama aroma nya white musk kalo kelama...,aku punya yg versi roll on jadi kayak oil yg ...,sangaaaatttt suka varian yang vegan ini aku...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,s o t b mengusung wangi citrus floral hangat ...,it s just tooo much for me buat yang suka par...,parfum yang sangat amat hitz di semua kalangan...,parfume yang wangi nya dar der dor parfume k...,cobain parfume dari saff co varian sotb ini ...,baru coba saff co edp sotb nya yg viral aku...,Notet teratas juga minum,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juice parfumny...,wangi yang bikin orang orang auto pada nengok ...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,aku beli perfume ini karena penasaran kaya gim...,ini koleksi parfume hmns ke ku pertama kali...,walopun aku cuma coba testernya tapi parfum ya...,pertama kali tau parfum ini dari kakak aku ak...,pertama kali coba hmns ya si orgasm ini awal ...,pertama kali cobain parfum nya hmns dan banya...,baunya skrg lebih enak drpd dulu waktu awal la...,parfume favorite aku wanginya tu bunga gitu ...,i got this as a gift from my friend pernah co...,best varian ever suka banhet sama wangi nya s...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,varian parfum evangeline yang paling aku suka ...,wanginya enak banget parfum ini menurut gua ke...,pertama kali cium langsung suka sama wanginya ...,ini parfum sejuta umat sih memang wanginya en...,dia tuh bau nya manis seger tapi strong gitu ...,wanginya enak banget menurut aku wanginya seg...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangeline ini yang paling norma...,black sakura wanginya unik banget soalnya dari...,hai girls beli ini karna racun banget harga...
...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [13]:
#create function untuk stemming
def text_stemming(text):
  # Tokenization
  tokens = word_tokenize(text)

  # Stopwords removal
  tokens = [word for word in tokens if word not in stopwords_id or stopwords_en]

  # Stemming
  tokens = [stemmer_id.stem(word) for word in tokens]
  tokens = [stemmer_en.stem(word) for word in tokens]
  # Combining Tokens
  text = ' '.join(tokens)

  return text

In [14]:
#stem processed review columns
for i in range(1,11):
  df[f'review_{i}_processed_tl_stem'] = df[f'review_{i}_processed_tl'].apply(lambda x: text_stemming(x))
df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_1_processed_tl_stem,review_2_processed_tl_stem,review_3_processed_tl_stem,review_4_processed_tl_stem,review_5_processed_tl_stem,review_6_processed_tl_stem,review_7_processed_tl_stem,review_8_processed_tl_stem,review_9_processed_tl_stem,review_10_processed_tl_stem
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,dari awal semprot udah rasa beda ada mani blac...,fatal adalah aroma sensual dan inten yang coco...,wangi enak laki juga masuk rekomendasi jugaa j...,wangi enak laki juga masuk rekomendasi jugaa j...,hint fatal extrait de parfum benar benar bikin...,wangi enak tahan lama gak bikin bau apek recom...,wangi sangat seductive dan kenceng banvwt sepe...,wangi enak bgt tahan lama veri recommend buat ...,wangi super wangi poll gabikin puse di kepala ...,wangi enak banget bikin seger dan ga bikin pus...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,damn dulu aku cium parfum ini slalu kek gaksuk...,udah gatau botol berapa karena selalu repurcha...,produk yang vegan dan alkohol free itu yang pa...,awal iseng mampir toko cium aroma powder yg so...,parfum suka ibu akuu wangi enak tru lumayan aw...,suka sama wangi muski powder floral lembut duh...,wangi enak bgt soft gitu ga bikin puse tahan j...,sukaa si sama aroma nya white musk kalo lama a...,aku punya yg versi roll on jadi kayak oil yg d...,sangaaaatttt suka varian yang vegan ini aku le...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,s o t b usung wangi citru floral hangat top no...,it s just tooo much for me buat yang suka parf...,parfum yang sangat amat hitz di semua kalang w...,parfum yang wangi nya dar der dor parfum kha c...,cobain parfum dari saff co varian sotb ini yg ...,baru coba saff co edp sotb nya yg viral aku su...,notet ata juga minum,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juic parfum jug...,wangi yang bikin orang orang auto pada nengok ...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,aku beli perfum ini karena penasaran kaya gima...,ini koleksi parfum hmn ke ku pertama kali snif...,walopun aku cuma coba tester tapi parfum yang ...,pertama kali tau parfum ini dari kakak aku aku...,pertama kali coba hmn ya si orgasm ini awal be...,pertama kali cobain parfum nya hmn dan banyak ...,bau skrg lebih enak drpd dulu waktu awal launc...,parfum favorit aku wangi tu bunga gitu kayak a...,i got thi as a gift from my friend pernah coba...,best varian ever suka banhet sama wangi nya se...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,varian parfum evangelin yang pale aku suka wan...,wangi enak banget parfum ini turut gua kek sod...,pertama kali cium langsung suka sama wangi dia...,ini parfum juta umat sih memang wangi enak ban...,dia tuh bau nya mani seger tapi strong gitu in...,wangi enak banget turut aku wangi seger mani g...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangelin ini yang pale normal a...,black sakura wangi unik banget soal dari bahan...,hai girl beli 

In [15]:
#replace row yang tidak memiliki text menjadi missing value
df.replace('', np.nan,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   link_product                 1020 non-null   object 
 1   product_image                1020 non-null   object 
 2   brand_name                   1020 non-null   object 
 3   product_name                 1020 non-null   object 
 4   product_variant              1020 non-null   object 
 5   full_name_product            1020 non-null   object 
 6   ratings                      1020 non-null   float64
 7   total_review                 1020 non-null   int64  
 8   price                        1020 non-null   int64  
 9   review_1                     1020 non-null   object 
 10  review_2                     1020 non-null   object 
 11  review_3                     1020 non-null   object 
 12  review_4                     1020 non-null   object 
 13  review_5          

In [16]:
#concate all stemmed reviews
df['review_processed'] = df['review_1_processed_tl_stem'] + df['review_2_processed_tl_stem'] + df['review_3_processed_tl_stem'] + df['review_4_processed_tl_stem'] +df['review_5_processed_tl_stem'] + df['review_6_processed_tl_stem'] + df['review_7_processed_tl_stem'] + df['review_8_processed_tl_stem'] +df['review_9_processed_tl_stem'] + df['review_10_processed_tl_stem']

df

,link_product,product_image,brand_name,product_name,product_variant,full_name_product,ratings,total_review,price,review_1,...,review_2_processed_tl_stem,review_3_processed_tl_stem,review_4_processed_tl_stem,review_5_processed_tl_stem,review_6_processed_tl_stem,review_7_processed_tl_stem,review_8_processed_tl_stem,review_9_processed_tl_stem,review_10_processed_tl_stem,review_processed
0,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HINT,Fatale Extrait de Parfum,-,Fatale Extrait de Parfum,4.2,894,249000,Dari awal semprot udah kerasa beda: ada manisn...,...,fatal adalah aroma sensual dan inten yang coco...,wangi enak laki juga masuk rekomendasi jugaa j...,wangi enak laki juga masuk rekomendasi jugaa j...,hint fatal extrait de parfum benar benar bikin...,wangi enak tahan lama gak bikin bau apek recom...,wangi sangat seductive dan kenceng banvwt sepe...,wangi enak bgt tahan lama veri recommend buat ...,wangi super wangi poll gabikin puse di kepala ...,wangi enak banget bikin seger dan ga bikin pus...,dari awal semprot udah rasa beda ada mani blac...
1,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,The Body Shop,White Musk Eau de Parfum,-,White Musk Eau de Parfum,4.3,604,329000,Damn dulu aku cium parfum ini slalu kek gaksuk...,...,udah gatau botol berapa karena selalu repurcha...,produk yang vegan dan alkohol free itu yang pa...,awal iseng mampir toko cium aroma powder yg so...,parfum suka ibu akuu wangi enak tru lumayan aw...,suka sama wangi muski powder floral lembut duh...,wangi enak bgt soft gitu ga bikin puse tahan j...,sukaa si sama aroma nya white musk kalo lama a...,aku punya yg versi roll on jadi kayak oil yg d...,sangaaaatttt suka varian yang vegan ini aku le...,damn dulu aku cium parfum ini slalu kek gaksuk...
2,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Saff & Co.,S.O.T.B Extrait de Parfum,-,S.O.T.B Extrait de Parfum,4.1,568,220000,"S.O.T.B mengusung wangi citrus-floral hangat, ...",...,it s just tooo much for me buat yang suka parf...,parfum yang sangat amat hitz di semua kalang w...,parfum yang wangi nya dar der dor parfum kha c...,cobain parfum dari saff co varian sotb ini yg ...,baru coba saff co edp sotb nya yg viral aku su...,notet ata juga minum,best parfum kala kalian suka dengan tipe wangi...,saff n co ini spl nya oke juga juic parfum jug...,wangi yang bikin orang orang auto pada nengok ...,s o t b usung wangi citru floral hangat top no...
3,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Orgasm,-,Orgasm,4.0,515,298000,Aku beli perfume ini karena penasaran kaya gim...,...,ini koleksi parfum hmn ke ku pertama kali snif...,walopun aku cuma coba tester tapi parfum yang ...,pertama kali tau parfum ini dari kakak aku aku...,pertama kali coba hmn ya si orgasm ini awal be...,pertama kali cobain parfum nya hmn dan banyak ...,bau skrg lebih enak drpd dulu waktu awal launc...,parfum favorit aku wangi tu bunga gitu kayak a...,i got thi as a gift from my friend pernah coba...,best varian ever suka banhet sama wangi nya se...,aku beli perfum ini karena penasaran kaya gima...
4,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Evangeline,Eau De Perfume Sakura Series,Black Sakura,Eau De Perfume Sakura Series Black Sakura,4.1,442,30000,Varian parfum Evangeline yang paling aku suka....,...,wangi enak banget parfum ini turut gua kek sod...,pertama kali cium langsung suka sama wangi dia...,ini parfum juta umat sih memang wangi enak ban...,dia tuh bau nya mani seger tapi strong gitu in...,wangi enak banget turut aku wangi seger mani g...,saya beli parfum ini dengan harga yang sangat ...,varian parfum evangelin ini yang pale normal a...,black sakura wangi unik banget soal dari bahan...,hai girl beli ini karna racun banget harga mur...,varian parfum evangelin 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   link_product                 1020 non-null   object 
 1   product_image                1020 non-null   object 
 2   brand_name                   1020 non-null   object 
 3   product_name                 1020 non-null   object 
 4   product_variant              1020 non-null   object 
 5   full_name_product            1020 non-null   object 
 6   ratings                      1020 non-null   float64
 7   total_review                 1020 non-null   int64  
 8   price                        1020 non-null   int64  
 9   review_1                     1020 non-null   object 
 10  review_2                     1020 non-null   object 
 11  review_3                     1020 non-null   object 
 12  review_4                     1020 non-null   object 
 13  review_5          

In [54]:
df_eda = df[['brand_name','product_name','product_variant','full_name_product','price','ratings','total_review','review_processed']]
df_eda.to_csv('eda.csv',index=False)

# Data Encoding

In [7]:
#using multi lingual embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [8]:
embedding = model.encode(df['review_processed'],convert_to_tensor=True)

In [9]:
#make recommender system function
def recommend_items(query,top_n=5):
  query_embedding = model.encode([query], convert_to_tensor=True)
  similarities = cosine_similarity(query_embedding, embedding)[0]

  # Get top N indices (excluding the input query itself if it's from the dataset)
  top_indices = similarities.argsort()[::-1][:top_n]
  results = df.iloc[top_indices].copy()
  results["similarity_score"] = similarities[top_indices]
  return results

In [11]:
#test recommender system
query = "aroma kaya kayu, cocok untuk laki dan perempuan, baunya kuat"
recommendations = recommend_items(query)
recommendations[['link_product','product_image','brand_name','full_name_product','price','review_processed']]

,link_product,product_image,brand_name,full_name_product,price,review_processed
555,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,ZARA,Energetically New York,499000,jujur yang ini rasa seperti saya tidak ada usa...
154,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,Alchemist Fragrance,Out West Extrait De Parfum,319000,im sorri wangi ga terlalu nyagrak tapi turut a...
495,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/300/remaster...,HMNS Perfume,Untitled Humans,395000,wangi enak soft dan seger tahan lama samp male...
854,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Morris,Lifestyle Edition Creative,45000,aroma sangat maskulin karena seperti ada aroma...
126,https://reviews.femaledaily.com/products/fragr...,https://image.femaledaily.com/dyn/210/images/p...,Coach,Coach new york EDP,1390000,wangi itu centil tapi independen dari jauh ras...


As we can see, the program can in fact recommend perfumes using sentences/keywords. Now we just have to save the model and the dataframe to be deployed

In [12]:
#save model
np.save("embedding.npy",embedding.cpu().numpy())
model.save('model_saved')

In [13]:
#zip file agar dapat di-download
shutil.make_archive('model_saved','zip','model_saved')

'/content/model_saved.zip'